In [77]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
import pandas as pd
import vaex as vx
from sklearn.metrics.pairwise import cosine_similarity
import requests
import shutil
from PIL import Image
import config

## Preprocessing steps

In [2]:
dfl = pd.read_csv('Data/link.csv')
dfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [3]:
dfl.dropna(inplace=True)
dfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27026 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27026 non-null  int64  
 1   imdbId   27026 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 844.6 KB


In [4]:
dfm = pd.read_csv('Data/movie.csv')
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [5]:
df_tmid=dfm.merge(dfl,on='movieId').drop(['imdbId'],axis=1)
df_tmid.head()

,movieId,title,genres,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,31357.0
4,5,Father of the Bride Part II (1995),Comedy,11862.0


In [6]:
dfr = pd.read_csv('Data/rating.csv')
dfr.drop(['timestamp'],axis=1,inplace=True)
dfr.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [7]:
preFinalDf= df_tmid.merge(dfr,on='movieId').drop(['movieId'],axis=1)
preFinalDf.head()

,title,genres,tmdbId,userId,rating
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,3,4.0
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,6,5.0
2,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,8,4.0
3,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,10,4.0
4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862.0,11,4.5


In [8]:
df = preFinalDf['genres']
a =[]
for i in df.unique():
    b=[word for word in i.split('|')]
    a.extend(b)
    
print(len(set(a)))

20


In [9]:
%%time
print(set(a))

{'Action', 'Horror', 'Musical', 'Crime', 'Comedy', 'Sci-Fi', 'Children', 'War', 'Film-Noir', '(no genres listed)', 'Western', 'IMAX', 'Drama', 'Mystery', 'Documentary', 'Animation', 'Adventure', 'Thriller', 'Fantasy', 'Romance'}
CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
total_genres = 18 # removed Film-Noir && (no genres listed)

In [19]:
%%time
vu=preFinalDf.groupby('userId').count()['rating']>600
validUsers= vu[vu].index

CPU times: total: 3 s
Wall time: 3.01 s


In [20]:
validUsers.shape

(5325,)

In [21]:
ppDf = preFinalDf[preFinalDf['userId'].isin(validUsers)]
ppDf.shape

(5396026, 5)

In [22]:
vm=ppDf.groupby('tmdbId').count()['rating']>500
validMovie = vm[vm].index

In [23]:
validMovie.shape

(2828,)

In [24]:
ppDf2 = ppDf[ppDf['tmdbId'].isin(validMovie)]
ppDf2.shape

(4098762, 5)

In [25]:
ppDf2.drop(['title','genres'],axis=1, inplace=True)
ppDf2.head()

C:\Users\abhinav kumar\AppData\Local\Temp\ipykernel_4260\3672254860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppDf2.drop(['title','genres'],axis=1, inplace=True)


,tmdbId,userId,rating
18,862.0,54,4.0
19,862.0,58,5.0
27,862.0,91,4.0
37,862.0,116,3.0
56,862.0,156,5.0


In [26]:
ppDf2 = ppDf2.astype({"tmdbId": int})
ppDf2.head()

,tmdbId,userId,rating
18,862,54,4.0
19,862,58,5.0
27,862,91,4.0
37,862,116,3.0
56,862,156,5.0


In [27]:
finalDf = ppDf2.pivot_table(index='tmdbId',columns='userId',values='rating')
finalDf.head()

userId,54,58,91,104,116,156,208,247,271,294,...,138211,138254,138270,138301,138307,138325,138382,138397,138406,138437
tmdbId,,,,,,,,,,,,,,,,,,,,,
5,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,4.5,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
11,4.0,NaN,4.5,3.0,4.5,NaN,4.5,4.0,2.5,4.0,...,4.0,4.5,4.0,3.5,2.0,5.0,4.0,3.5,4.0,5.0
12,NaN,NaN,4.0,NaN,3.0,NaN,3.5,NaN,2.5,4.5,...,4.5,4.0,NaN,3.5,NaN,4.0,NaN,NaN,4.0,4.0
13,5.0,4.5,4.0,NaN,4.0,5.0,1.0,4.0,3.5,4.5,...,5.0,4.0,4.0,3.5,NaN,4.5,5.0,4.5,4.0,NaN
14,5.0,4.5,4.5,2.0,4.5,5.0,3.5,4.0,NaN,3.5,...,5.0,4.0,5.0,5.0,4.0,5.0,3.0,4.0,4.0,5.0


In [28]:
finalDf.fillna(0,inplace=True)
finalDf.shape

(2828, 5325)

In [29]:
finalDf.head()

userId,54,58,91,104,116,156,208,247,271,294,...,138211,138254,138270,138301,138307,138325,138382,138397,138406,138437
tmdbId,,,,,,,,,,,,,,,,,,,,,
5,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,4.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11,4.0,0.0,4.5,3.0,4.5,0.0,4.5,4.0,2.5,4.0,...,4.0,4.5,4.0,3.5,2.0,5.0,4.0,3.5,4.0,5.0
12,0.0,0.0,4.0,0.0,3.0,0.0,3.5,0.0,2.5,4.5,...,4.5,4.0,0.0,3.5,0.0,4.0,0.0,0.0,4.0,4.0
13,5.0,4.5,4.0,0.0,4.0,5.0,1.0,4.0,3.5,4.5,...,5.0,4.0,4.0,3.5,0.0,4.5,5.0,4.5,4.0,0.0
14,5.0,4.5,4.5,2.0,4.5,5.0,3.5,4.0,0.0,3.5,...,5.0,4.0,5.0,5.0,4.0,5.0,3.0,4.0,4.0,5.0


In [30]:
finalDf.to_csv('recomderData.csv')

In [37]:
vax_df = vx.from_pandas(finalDf)

In [38]:
vax_df.export_hdf5('recomderData.hdf5')

In [39]:
%%time
df = vx.open('recomderData.hdf5')
df

CPU times: total: 2.41 s
Wall time: 2.59 s


#,54,58,91,104,116,156,208,247,271,294,298,347,348,359,367,370,388,394,413,424,572,586,587,609,614,631,632,637,648,702,710,717,741,768,775,777,812,856,903,910,955,971,982,1011,1052,1113,1165,1185,1200,1244,1296,1323,1411,1419,1421,1507,1516,1560,1568,1588,1602,1644,1678,1715,1729,1741,1748,1755,1763,1849,1854,1864,1887,1906,1908,1931,1942,1966,1972,2024,2038,2047,2071,2072,2099,2113,2158,2161,2171,2232,2242,2261,2311,2344,2349,2352,2374,2378,2397,2398,...,135808,135819,135828,135888,135976,135984,135993,136015,136053,136054,136082,136132,136151,136187,136223,136262,136268,136293,136301,136426,136479,136521,136564,136568,136578,136595,136599,136673,136679,136684,136754,136806,136819,136860,136875,136877,136896,136897,136940,136941,136946,136957,136989,136997,137015,137031,137037,137130,137164,137202,137277,137280,137342,137343,137366,137369,137377,137387,137417,137501,137525,137558,137563,137683,137686,137717,137746,137805,137807,137838,137839,137854,137869,137885,137893,137934,137949,137976,137977,137993,138019,138072,138103,138134,138146,138148,138162,138164,138200,138208,138211,138254,138270,138301,138307,138325,138382,138397,138406,138437
0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,3.5,2.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.0,1.5,0.0,2.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,0.0,3.5,3.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,1.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.5,4.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,5.0,3.5,4.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,4.0,0.0,1.0,0.0,4.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
1,4.0,0.0,4.5,3.0,4.5,0.0,4.5,4.0,2.5,4.0,5.0,4.0,0.0,5.0,4.0,0.0,3.0,4.0,5.0,3.5,0.0,3.5,4.0,4.5,4.0,4.5,3.0,4.0,2.0,4.5,5.0,4.0,5.0,4.5,5.0,2.0,4.0,0.0,5.0,4.5,4.0,5.0,4.0,5.0,4.5,0.0,4.0,4.5,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,2.5,4.5,0.0,4.0,3.5,4.0,3.0,4.0,4.0,4.0,4.0,4.5,4.0,4.0,3.5,5.0,3.0,0.0,5.0,4.0,5.0,4.0,4.5,4.0,4.0,4.0,4.5,2.0,4.0,4.5,5.0,4.5,4.0,5.0,5.0,5.0,5.0,2.5,3.0,5.0,4.0,4.0,...,4.0,0.5,3.5,0.0,5.0,1.0,4.5,0.0,3.5,4.5,5.0,5.0,4.0,3.0,0.0,5.0,4.0,4.5,4.0,4.0,2.0,5.0,4.0,5.0,0.0,4.5,4.0,4.5,0.0,0.0,4.0,4.5,0.0,5.0,4.0,5.0,5.0,4.0,5.0,2.0,3.5,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.5,4.0,0.0,5.0,5.0,4.0,5.0,3.0,4.0,4.0,5.0,3.5,3.0,5.0,4.0,5.0,3.5,4.0,4.0,4.0,4.0,5.0,4.5,5.0,4.0,4.0,5.0,3.0,4.0,4.0,4.5,5.0,4.0,4.0,5.0,0.0,3.0,5.0,3.5,0.0,3.0,4.0,4.5,4.0,3.5,2.0,5.0,4.0,3.5,4.0,5
2,0.0,0.0,4.0,0.0,3.0,0.0,3.5,0.0,2.5,4.5,0.0,3.5,4.5,3.0,0.0,2.5,0.0,0.0,4.0,4.5,4.0,0.0,4.5,3.0,3.5,4.0,0.0,3.5,0.0,4.0,4.5,0.0,4.0,0.0,5.0,4.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,4.0,4.0,4.0,0.0,4.0,4.5,4.5,4.0,3.5,0.0,5.0,3.0,4.0,4.5,4.0,4.5,4.0,0.0,2.5,4.5,4.5,4.0,4.5,4.0,0.0,4.5,4.5,4.5,0.0,0.0,4.0,0.0,4.0,4.5,5.0,3.5,3.0,4.5,0.0,4.5,5.0,4.0,3.5,0.0,4.0,0.0,1.5,5.0,1.5,0.0,0.0,4.0,0.0,...,4.5,0.0,3.0,0.0,4.5,4.5,4.5,4.0,2.5,0.0,5.0,4.5,0.0,4.5,4.5,5.0,3.5,0.0,4.0,4.0,4.0,3.5,4.0,0.0,0.0,3.0,4.0,3.5,0.0,4.0,4.0,0.0,4.5,3.0,3.5,5.0,0.0,4.0,0.0,0.0,3.0,4.5,1.0,0.0,3.5,1.5,0.0,4.0,4.0,4.0,4.0,0.0,0.0,5.0,0.0,0.0,4.5,0.0,5.0,0.0,4.0,0.0,0.0,4.0,4.5,3.5,2.5,4.0,0.0,4.5,3.5,4.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,3.5,0.0,4.0,5.0,0.0,2.5,4.0,4.5,4.0,0.0,3.5,0.0,4.0,0.0,0.0,4.0,4
3,5.0,4.5,4.0,0.0,4.0,5.0,1.0,4.0,3.5,4.5,3.0,4.0,0.0,4.5,4.0,5.0,3.0,0.0,0.0,3.0,5.0,3.5,5.0,2.0,3.5,4.0,2.5,3.5,3.0,3.5,0.0,3.5,5.0,4.0,4.0,3.5,4.0,0.0,5.0,4.0,5.0,4.0,2.5,3.0,3.0,0.0,3.5,4.5,5.0,5.0,3.0,4.5,5.0,4.0,2.5,5.0,4.5,4.0,3.0,4.0,4.0,4.0,4.5,4.0,3.0,4.0,3.5,3.0,0.0,4.0,4.5,4.0,4.0,3.5,3.0,5.0,5.0,4.0,4.0,4.5,4.0,5.0,4.0,3.0,0.5,3.0,5.0,5.0,3.0,2.5,4.5,4.0,4.0,4.5,5.0,0.5,4.0,5.0,4.0,4.0,...,2.0,4.0,3.0,1.0,3.0,3.

## Predict similar movies when one movie is provided using cosine similarity

In [41]:
movieSimilarity = cosine_similarity(finalDf)

In [60]:
def getSimilarMovie(movieNo):
    a=sorted(list(enumerate(movieSimilarity[movieNo])),key= lambda x:x[1], reverse=True)[1:6]
    l=[]
    for i in a:
        l.append(finalDf.index[i[0]])
    return l

In [80]:
pd.DataFrame(movieSimilarity).to_csv('cosineSimilarityMovie.csv')

In [61]:
getSimilarMovie(5)

[289, 935, 275, 238, 567]

In [78]:
pURL = "https://api.themoviedb.org/3/movie/"
endUrl = "?api_key="+config.apiKey+"&language=en-US"
URL = pURL+str(935)+endUrl
r = requests.get(url = URL)
data = r.json()
iUrl ="http://image.tmdb.org/t/p/w500"+data['poster_path']
ir = requests.get(url=iUrl,stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(ir.raw, out_file)

## Using NN for describing movie content